# Advection Diffusion Solver 


## Problem statement

We are investigating the Advection Diffusion Equation
$$
  \frac{\partial u}{\partial t} +\vec{u} \cdot \nabla u - \mu \nabla^2 u + S(u)=0 \in (0,2)^2
$$
with the initial value
$$
    u_0(x,y) = \sin(a x) \sin(b y),
$$
and the manufactured solution
$$
u^{\mathrm{MS}}(x,y,t) = \sin(a x) \sin(b y) \cos(c t)
$$
(which is directly emposed on the boundary)
The parameters are chose
$$
\mu=0.5,~ a = 2.3,~b = 2.9, ~c = 4.2, ~\vec{u}=(0.8 ~0.6)^T
$$

#### Calculation of the Source Term
The Source Term is computed such that $u^\text{MS}$ satisfies the equation, i.e. 
$$
S(u) = -(\frac{\partial u^{\mathrm{MS}}}{\partial t} +\vec{u} \cdot \nabla u^{\mathrm{MS}} - \mu \nabla^2 u^{\mathrm{MS}}) 
$$

We compute it by computing all parts
$$ 
\frac{\partial u^{\mathrm{MS}}}{\partial t} = -c \sin(a x) \sin(b y) \sin(c t)%= -c\frac{\sin(c t)}{\cos(c t)} u^{\mathrm{MS}}
$$
$$
\vec{u} \cdot \nabla u^{\mathrm{MS}} = \cos(ct) [u_1 a \cos(ax) \sin(by)+ u_2 b \sin(ax) \cos(by)] %=  u_1 a  \frac{\cos(ax)}{\sin(a x)}  u^{\mathrm{MS}}+ u_2 b \frac{\cos(by)}{\sin(by)}  u^{\mathrm{MS}}
$$
$$
\mu \nabla^2 u^{\mathrm{MS}} = \mu \cos(ct)[-a^2 \sin(ax) \sin(by)  - b^2 \sin(ax) \sin(by)]% =  \mu (-a^2 - b^2)u^{\mathrm{MS}}
$$



In [90]:
#r ".\binaries\\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 204
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#91.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [91]:
using BoSSS.Solution.XdgTimestepping;

## Creation of Differential Operator


Velocity field and Exact solution for Dirichlet Boundary

In [92]:
using static System.Math;
public static class MyGlobals {
    public static double u(double[] X) => 0.8;
    public static double v(double[] X) => 0.6;
    public static double a = 2.3;
    public static double b = 2.9;
    public static double c = 4.2;
    public static double nu => !isHeat ?  0.5:1.0;
    public static bool isHeat=false;
    public static double endTime=> !isHeat ?  2.0:0.1;
    public static double uExact(double[] X, double t) => !isHeat? Sin(a * X[0]) * Sin(b * X[1]) * Cos(c * t) : Math.Cos(X[0]*Math.PI*0.5)*Math.Cos(X[1]*Math.PI*0.5) ;

    public static double TimeDerivative(double[] X, double t) => -c * Sin(a * X[0]) * Sin(b * X[1]) * Sin(c * t);

    public static double ConvectiveTerm(double[] X, double t) => 
    Cos(c * t) * (u(X) * a * Cos(a * X[0]) * Sin(b * X[1]) + v(X) * b * Sin(a * X[0]) * Cos(b * X[1]));
    
    

    public static double DiffusionTerm(double[] X, double t) => nu * Cos(c * t) * (-a*a * Sin(a * X[0]) * Sin(b * X[1]) - b*b * Sin(a * X[0]) * Sin(b * X[1]));

     public static double SourceTerm (double[] X, double t) =>  !isHeat? -(TimeDerivative(X,t)+ConvectiveTerm(X,t)-DiffusionTerm(X,t)) : 0;
  
//     public static double TimeDerivative_u(double U,double[] X, double t) => -c *  Sin(c * t)*U/Cos(c*t);

//     public static double ConvectiveTerm_u(double U,double[] X, double t) => 
//    (u(X) * a * Cos(a * X[0]) /Sin(a*X[0])+ v(X) * b * Cos(b * X[1])/Sin(b*X[1]) )*U;

//     public static double DiffusionTerm_u(double U,double[] X, double t) => nu *(-a*a-b*b)*U;
//     // Calculate the corresponding source term S(u)
//     public static double S_u (double U,double[] X, double t) =>  -(TimeDerivative_u(U,X,t)+ConvectiveTerm_u(U,X,t)-DiffusionTerm_u(U,X,t));
   
}

Regarding the spatial part of the operator, we re-use the SIP-implementation 
which is already available in the `BoSSS.Solution.NSECommon` library.
For this implementation, whe have to specify the diffusion coefficient (here to -0.5)
as well as the Location of the Dirichlet boundary.

In [93]:
class Laplace : BoSSS.Solution.NSECommon.SIPLaplace {

    public Laplace() : base(1.4, "u") { } // override the constuctor; 
    //                                       The factor 1.4 is a constant "safty factor" multiper for the penalty parameter.
    //                                       The base implementation takes care about all other penalty factor dependencies, 
    //                                       i.e. dependence of the penatly factor on local cell size, DG polynomial degree and cell shape.
    //                                       The second argument, "u", is the name of the variable for the `ArgumentOrdering`.


    // Specifies whetherss a specific point (`inp.X`) is either a Dirichlet or a Neumann boundary;
    // Since in our case ther, the entire boundary should be Diriclet, we always return true.
    protected override bool IsDirichlet(ref CommonParamsBnd inp) {
        return true;
    }
    
    // diffusion coefficient
    override public double Nu(double[] x, double[] p, int jCell) {
        return -MyGlobals.nu;
    }
    override protected double g_Diri(ref CommonParamsBnd inp) { 
        //Console.WriteLine("Evaluated at x=" + inp.X);
        return !MyGlobals.isHeat? MyGlobals.uExact(inp.X,inp.time):0.0;
        //return 0;
    }
}

#### Advection Operator

The flux we are considering is the local Lax-Friedrichs flux
$$
\hat{f}(u_h^-, u_h^+,  \vec{n}) = \{u_h\} \vec{u} \cdot \vec{n} + C \lbrack \lbrack {u_h} \rbrack \rbrack.
$$
The constant $C \in \mathbb{R}^+$ has to be sufficiently large in order to guarantee the stability of the 
numerical scheme. We choose 
$$
C = \max \vert{\vec{u} \cdot \vec{n}}\vert 
$$


In [94]:
//double C = 0.8;     
 
Func<double, double, Vector, Vector, double> laxFriedrichsFlux =     
    delegate(double Uin, double Uout, Vector n, Vector velocityVector) {     
        var C=Math.Abs(velocityVector*n);
        return 0.5 * (Uin + Uout) * velocityVector * n - C * (Uout - Uin);     
    };

In [95]:

class ScalarTransportFlux : IEdgeForm, IVolumeForm
{
    public TermActivationFlags BoundaryEdgeTerms => TermActivationFlags.UxV;

    public TermActivationFlags InnerEdgeTerms => TermActivationFlags.UxV;

    public TermActivationFlags VolTerms => TermActivationFlags.UxGradV;

    public IList<string> ParameterOrdering => null;
     
    public IList<string> ArgumentOrdering {     
        get { return new string[] { "u" }; }     
    }  

    private Func<double, double, Vector, Vector, double> numericalFlux;     
 
    // Provides instances of this class with a specific flux implementation     
    public ScalarTransportFlux(     
        Func<double, double, Vector, Vector, double> numericalFlux) {     
 
        this.numericalFlux = numericalFlux;     
    }     
   
 
    protected void Flux(     
        double time, double[] x, double[] U, double[] output) {     
 
        output[0] = MyGlobals.u(x) * U[0];     
        output[1] = MyGlobals.v(x) * U[0];     
    }     
 
    // Makes use of the flux implementation supplied in the constructor     
    protected  double InnerEdgeFlux(     
        double time, double[] x, double[] normal,      
        double[] Uin, double[] Uout, int jEdge) {     
 
        Vector n              = new Vector(normal);     
        Vector velocityVector = new Vector(MyGlobals.u(x), MyGlobals.v(x));     
 
        return numericalFlux(Uin[0], Uout[0], n, velocityVector);     
    }     
 
    protected  double BorderEdgeFlux(     
        double time, double[] x, double[] normal, byte EdgeTag,     
        double[] Uin, int jEdge) {     
 
        double[] Uout = new double[] { MyGlobals.uExact(x, time) };     
 
        return InnerEdgeFlux(time, x, normal, Uin, Uout, jEdge);     
    }   
    public double VolumeForm(ref CommonParamsVol cpv, double[] U, double[,] GradU, double V, double[] GradV)
    {

        return - MyGlobals.u(cpv.Xglobal) * U[0]*GradV[0]-MyGlobals.v(cpv.Xglobal) * U[0]*GradV[1];
    }  
        public double BoundaryEdgeForm(ref CommonParamsBnd inp, double[] _uA, double[,] _Grad_uA, double _vA, double[] _Grad_vA)
    {
        return BorderEdgeFlux(     
        inp.time, inp.X, inp.Normal, inp.EdgeTag,     
        _uA, inp.iEdge)*_vA;
    }

    public double InnerEdgeForm(ref CommonParams inp, double[] _uIN, double[] _uOUT, double[,] _Grad_uIN, double[,] _Grad_uOUT, double _vIN, double _vOUT, double[] _Grad_vIN, double[] _Grad_vOUT)
    {
        return InnerEdgeFlux(inp.time, inp.X, inp.Normal,     
        _uIN,_uOUT, inp.iEdge)*(_vIN-_vOUT);
    }
}

In [96]:
class SourceTerm : IVolumeForm
{

    public TermActivationFlags VolTerms => TermActivationFlags.V;

    public IList<string> ArgumentOrdering {     
        get { return new string[] { "u" }; }     
    } 

    public IList<string> ParameterOrdering => null;

    public double VolumeForm(ref CommonParamsVol cpv, double[] U, double[,] GradU, double V, double[] GradV)
    {
        //return U[0]*V;
        //return MyGlobals.S_u(U[0],cpv.Xglobal,cpv.time)*V;
        return MyGlobals.SourceTerm(cpv.Xglobal,cpv.time)*V;
    }
}

#### Operator assembly
We compose the differential operator from the previously created 
`Laplace` implementation and add the `ScalarTransportFlux`. 

In [97]:
var Op = new DifferentialOperator(1, 0, 1, QuadOrderFunc.Linear(), "u", "R1");

Op.EquationComponents["R1"].Add(new Laplace()); // adding Laplace
if(MyGlobals.isHeat){
}else{
    Op.EquationComponents["R1"].Add(new ScalarTransportFlux(laxFriedrichsFlux)); // adding Advection
    Op.EquationComponents["R1"].Add(new SourceTerm()); // adding source term
}

Op.TemporalOperator = new ConstantTemporalOperator(Op);
Op.IsLinear = true;
Op.Commit();

In [98]:
Op.EquationComponents.Count()

1

## Find the time step

## Performing Temporal Integration

In [99]:

SinglePhaseField TimeIntegrate(SinglePhaseField u0, double EndTime, double dt, TimeSteppingScheme tsScheme=TimeSteppingScheme.ESDIRK_64) {
    var u1 = u0.CloneAs();
    var Timestepper = new XdgTimestepping(Op, new DGField[] { u1 }, new DGField[] { new SinglePhaseField(u0.Basis)}, 
                            tsScheme);

    int NoOfTimeteps = (int) (EndTime/dt);
    for(int i = 0; i < NoOfTimeteps; i++)
        Timestepper.Solve(dt*i, dt);
    
    return u1;
}
SinglePhaseField TimeIntegrate(SinglePhaseField u0, double EndTime, int NoOfTimeteps, TimeSteppingScheme tsScheme=TimeSteppingScheme.ESDIRK_64) {
    var u1 = u0.CloneAs();
    var Timestepper = new XdgTimestepping(Op, new DGField[] { u1 }, new DGField[] { new SinglePhaseField(u0.Basis)}, 
                            tsScheme);

    double dt = EndTime/NoOfTimeteps;
    for(int i = 0; i < NoOfTimeteps; i++)
        Timestepper.Solve(dt*i, dt);
    
    return u1;
}

## Performing a temporal convergence study

In [100]:
double uExEq(double[] X) {
   double Lambda = -Math.PI.Pow2()*0.5;
   double tend = 0.1;
   double u0 = Math.Cos(X[0]*Math.PI*0.5)*Math.Cos(X[1]*Math.PI*0.5);
   return  !MyGlobals.isHeat? MyGlobals.uExact(X,MyGlobals.endTime) : Math.Exp(Lambda*MyGlobals.endTime)*u0;
}

loop over different grids (`nCells`) and `TimesteppingSchemes` 

In [101]:
!MyGlobals.isHeat

True

In [102]:
//var dtS = new double[] {1,0.5,0.25,0.125,0.0625,0.03175,0.015875,0.0079375};
bool debug=false;
var dtS = new double[] {1,0.5,0.25,0.125,0.0625};
var NoOfTimestepS = new int[] { 1, 2, 4, 8, 16,32 /*, 64, 128, 256, 512, 1028*/ };
var NCells = new double[] {4};
//var tsSchemes= new TimeSteppingScheme[] {TimeSteppingScheme.ESDIRK_64,TimeSteppingScheme.ESDIRK_32};
// var NCells = new double[] {4,8,16,32,64,128};
// var dtS = new double[] {1,0.5};
var tsSchemes= new TimeSteppingScheme[] {TimeSteppingScheme.ESDIRK_64};
var results = new List<Tuple<string,double[]>>();

//Grid and initial value
foreach(int nCells in NCells){
    Grid2D grid;
    if(!MyGlobals.isHeat){
        grid = Grid2D.Cartesian2DGrid(GenericBlas.Linspace(0.0,2.0,nCells+1), GenericBlas.Linspace(0.0,2.0,nCells+1));
    }else{
        grid = Grid2D.Cartesian2DGrid(GenericBlas.Linspace(-1.0,1.0,nCells+1), GenericBlas.Linspace(-1.0,1.0,nCells+1));
    }
    
    var u0 = new SinglePhaseField(new Basis(grid, 2), "u");
    var u1 = new SinglePhaseField(new Basis(grid, 2), "u");
    u0.ProjectField((double[] X) => MyGlobals.uExact(X,0));
    //u0.ProjectField((double[] X) => Math.Cos(X[0]*Math.PI*0.5)*Math.Cos(X[1]*Math.PI*0.5));
    for(int ischeme=0;ischeme<tsSchemes.Length;ischeme++){
        var scheme=tsSchemes[ischeme];
        var solutions = new List<SinglePhaseField>();
        Console.WriteLine("   Starting Compuation: " + scheme.ToString()+": nCells=" +grid.NumberOfCells);

        //### This loop uses a fixed timestep prescribed on the basis fo NoOfTimestepS
        for(int i = 0; i < NoOfTimestepS.Length; i++) {
            Console.WriteLine("   --------------  computing no of timeteps: " + NoOfTimestepS[i]);
            u1 = TimeIntegrate(u0, MyGlobals.endTime, NoOfTimestepS[i],scheme);
            //Tecplot("u1." + i, u1);
            solutions.Add(u1);
        }

        //### This loop uses a fixed timestep prescribed by a value dt
        // for(int i = 0; i < dtS.Length; i++) {
        //     Console.WriteLine("   --------------  computing dt: " + dtS[i]);
        //     u1 = TimeIntegrate(u0, MyGlobals.endTime, dtS[i],scheme);

        //     //use Exact solution
        //     //u1.ProjectField((double[] X) => MyGlobals.uExact(X,MyGlobals.endTime));
        //     //Tecplot("u1." + i, u1);
        //     solutions.Add(u1);
        // }

        if(debug){
            //######## some debugging ####################
            //compute res
            var Ev= Op.GetEvaluatorEx(new List<DGField>(){u1},null,new CoordinateMapping(u1));
            var residual=u1.CloneAs(); residual.Identification="residual";
            Ev.Evaluate(1.0,0.0,new CoordinateVector(residual));
            
            //diff to exact sol
            var diff=u1.CloneAs(); diff.Identification="diff_ex";
            diff.ProjectField((double[] X) => MyGlobals.uExact(X,MyGlobals.endTime));
            diff.Acc(-1.0,u1);

            //exact soll
            var uex=u1.CloneAs(); uex.Identification="u_ex";
            uex.ProjectField((double[] X) => MyGlobals.uExact(X,MyGlobals.endTime));

            //plot
            Tecplot("u1." + ((ischeme+1)*100+nCells), new DGField[] {u1,residual,diff,uex});
        }

        //Computing the error against the exact solution:
        double[] ErrEx = new double[solutions.Count];
        for(int i = 0; i < ErrEx.Length; i++) {
            ErrEx[i] = solutions[i].L2Error(uExEq);
            //ErrEx[i]=residual.L2Norm();
        }
        results.Add(new Tuple<string,double[]>(scheme.ToString()+ ": nCells=" +grid.NumberOfCells,ErrEx));
    }
}



plot results

In [103]:
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Blue,LineColors.Black,LineColors.Black, LineColors.Red,LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

In [104]:
var gp = new Gnuplot();
//double[] ts = dtS.Take(dtS.Length).Select(x => (double)x).ToArray();
double[] ts = NoOfTimestepS.Take(NoOfTimestepS.Length).Select(x => MyGlobals.endTime/(double)x).ToArray();
for(int iRes=0;iRes<results.Count;iRes++){
    var pair= results[iRes];
    gp.PlotLogXLogY(ts, pair.Item2,  title:pair.Item1, format:GetFormat(iRes));
}
gp.PlotNow()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.01 
 
 
 
 
 0.1 
 
 
 
 
 1 
 
 
 
 
 0.01 
 
 
 
 
 0.1 
 
 
 
 
 1 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 ESDIRK64: nCells=16 
 
 
 ESDIRK 6 4: nCells=16

In [105]:
results[2].Item2

Error: System.ArgumentOutOfRangeException: Index was out of range. Must be non-negative and less than the size of the collection. (Parameter 'index')
   at System.Collections.Generic.List`1.get_Item(Int32 index)
   at Submission#106.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

### Spatial error estimation
To get an estimation of the spatial error, one needs a reference solution, i.e. a solution on a finer dimensional spatial approximation space. In the present work, we use an order-incremented finer space, $p\rightarrow p+1$, and define $\mathbf{U}_{p+1}=\mathbf{I}^p_{p+1}\mathbf{U}_{p}$ the prolongation of the coarse state to the finer space. The operator $\mathbf{I}^p_{p+1}$ in the present case is an $L_2$ projection. To measure the spatial error, the velocity on the coarse space is compared with the one on the fine space. This comparison must be done on the same approximation space, taken as $p+1$, and hence the coarse-space velocity is prolongated to the fine space
$$
    \varepsilon_{space} = ||\mathbf{R}_{p+1}(\mathbf{I}_{p+1}^p \mathbf{U}_p^{n+1}) - \mathbf{M}_{p+1}\mathbf{I}_{p+1}^p\mathbf{M}_{p}^{-1}\mathbf{R}_{p}(\mathbf{U}_p^{n+1})|| .
$$

`Note`: For SinglephaseFields in BoSSS the Mass Matrices are identity and the prolongation operator is a simple inclusion. Thus the the result is the norm of the enriched residual vector setting all entries corresponding to degree less or equal than p to zero.

In [106]:
using ilPSP.LinSolvers;
static DGField[] ProlongateCoarseState(DGField[] U)
{
    
    // Prolongating coarse state to the finer space
    var Up1= U.CloneAs();
    if(U is SinglePhaseField[] uSF){
        for (int iField=0;iField< U.Length;iField++){
            var enriched_basis= new Basis(U[iField].Basis.GridDat,U[iField].Basis.Degree+1);
            Up1[iField]=new SinglePhaseField(enriched_basis,U[iField].Identification);
            Up1[iField].AccLaidBack(1.0,U[iField]);
        }
    }else if(U is XDGField[] Ux){
        for (int iField=0;iField< U.Length;iField++){
            var enriched_basis= new XDGBasis(Ux[iField].Basis.Tracker,Ux[iField].Basis.Degree+1);
            Up1[iField]=new XDGField(enriched_basis,Ux[iField].Identification);
            Up1[iField].AccLaidBack(1.0,Ux[iField]);
        }
    }else{
        throw new NotSupportedException("this type of DGField is not supported");
    }
    return Up1;
}
static double[] EvalEnrichedRes(DGField[] U,IDifferentialOperator Op){
    var Up1= ProlongateCoarseState(U);
    return EvalRes(Up1,Op);
}
static double[] EvalRes(DGField[] U,IDifferentialOperator Op){
    var U_map = new CoordinateMapping(U);
    IEvaluatorNonLin Eval_R =null;
    if(U is SinglePhaseField[] uSF && Op is DifferentialOperator){ 
        Eval_R = Op.GetEvaluatorEx( U, null, U_map);
    }else if(U is XDGField[] Ux && Op is XDifferentialOperatorMk2 OpX){
        var LsTrk = Ux[0].Basis.Tracker;
        Eval_R = OpX.GetEvaluatorEx(LsTrk, U, null, U_map);
    }else{
        throw new NotSupportedException("this type of DGField is not supported");
    }
    double[] en_res=new double[U_map.TotalLength];
    Eval_R.Evaluate(1.0, 0.0, en_res);
    return en_res;
}
static DGField[] EvalRes_toDGField(DGField[] U,IDifferentialOperator Op){
    double[] RpUp=EvalRes(U,Op);
    DGField[] Residual=null;
    
    if(U is SinglePhaseField[] uSF){
        Residual = new SinglePhaseField[U.Length];
        for (int iField=0;iField< U.Length;iField++){
            Residual[iField]=new SinglePhaseField(U[iField].Basis,U[iField].Identification);
        }
    }else if(U is XDGField[] Ux){
        Residual = new XDGField[U.Length];
        for (int iField=0;iField< U.Length;iField++){
            Residual[iField]=new XDGField(Ux[iField].Basis,Ux[iField].Identification);
        }
    }else{
        throw new NotSupportedException("this type of DGField is not supported");
    }

    var res_vec= new CoordinateVector(Residual);
    res_vec.Acc(1.0,RpUp);
    return Residual;
}
static double CalculateDiffNorm(double[] first_term,double[] sec_term){
    if(first_term.Length==sec_term.Length){
        // calc the difference
        var diff=first_term.CloneAs();
        diff.AccV(-1.0,sec_term);
        return (diff.L2Norm());
    }else{
        throw new ArgumentException("lengths of input must match");
    }
}

static double[] CalculateSpatialErrorElementwise(DGField[] U,IDifferentialOperator Op)
{

    // first term
    var Up1=ProlongateCoarseState(U);
    var Rp1Up1=EvalRes_toDGField(Up1,Op);
    var Rp1Up1_vec= new CoordinateVector(Rp1Up1);
    Rp1Up1_vec.Clear();
    Rp1Up1_vec.SetV(EvalEnrichedRes(U,Op));

    //second term
    var RpUp=EvalRes_toDGField(U,Op);
    var sec_term = ProlongateCoarseState(RpUp);
    var sec_term_vec= new CoordinateVector(sec_term);

    //XDG Case we need to multiply mass matrices
    if(U is XDGField[] Ux){
        var res_vec= new CoordinateVector(RpUp);
        // get p Mass Mat and multiplicate with R_p(U_p)
        var LsTrk= Ux[0].Basis.Tracker;
        var mapping= new CoordinateMapping(U);
        MassMatrixFactory mMFp = LsTrk.GetXDGSpaceMetrics(LsTrk.SpeciesIdS, Op.GetOrderFromQuadOrderFunction(mapping.BasisS,null,mapping.BasisS)).MassMatrixFactory;
        BlockMsrMatrix Mp_inv = mMFp.GetMassMatrix(mapping, inverse: true);
        double[] MpxRp= new double[res_vec.Length];
        Mp_inv.SpMV(1.0,res_vec,0.0,MpxRp);

        //put result into CoordinateVector of RpUp
        res_vec.Clear();
        res_vec.Acc(1.0,MpxRp);

        //do the prolongation I_p*(...)
        var prol_state=ProlongateCoarseState(RpUp);

        // multpliy p+1 Mass Matrix
        var mapping_p1= new CoordinateMapping(prol_state);
        MassMatrixFactory mMFp1 = LsTrk.GetXDGSpaceMetrics(LsTrk.SpeciesIdS, Op.GetOrderFromQuadOrderFunction(mapping_p1.BasisS,null,mapping_p1.BasisS)).MassMatrixFactory;
        BlockMsrMatrix Mp1 = mMFp1.GetMassMatrix(mapping_p1, inverse: false);
        double[] Mp1xRp1= new double[res_vec.Length];

        sec_term_vec.Clear();
        Mp1.SpMV(1.0,new CoordinateVector( prol_state),0.0,sec_term_vec);
    }

    // Assemble the element-wise spatial error
    int nCells= U[0].Basis.GridDat.iGeomCells.Count;
    var epsilonSpace =new double[nCells];
    BitArray cells= new BitArray(U[0].Basis.GridDat.iGeomCells.Count);

    for (int cell=0;cell<nCells; cell++){ //loop over all cells
        double acc=0;
        cells[cell]=true;
        var cm= new CellMask(U[0].Basis.GridDat,cells);
        for (int iField=0;iField< U.Length;iField++){ //loop over Fields
            epsilonSpace[cell]=Rp1Up1[iField].L2Error(sec_term[iField],cm);
        };
        cells[cell]=false;
    }
    return epsilonSpace;
}


In [107]:
var nC=2;
var gridTest = Grid2D.Cartesian2DGrid(GenericBlas.Linspace(0.0,2.0,nC+1), GenericBlas.Linspace(0.0,2.0,nC+1));
var u = new SinglePhaseField(new Basis(gridTest, 2), "u");
u.ProjectField((double[] X) => MyGlobals.uExact(X,2.0));
var U =new SinglePhaseField[1]; U[0]=u;
CalculateSpatialErrorElementwise(U,Op)

[ 25.405865012044387, 110.36684353541315, 201.73773353375, 220.67661410404887 ]

In [108]:
static double ComputeL2NormFromIndex(double[] array, int j)
    {
        // Check if the array has sufficient length
        if (j >= array.Length)
        {
            throw new ArgumentException("Index j exceeds the length of the array.");
        }

        // Calculate the squared sum of squares for elements starting from index j
        double sumOfSquares = 0;
        for (int i = j; i < array.Length; i++)
        {
            sumOfSquares += array[i] * array[i];
        }

        // Calculate the square root of the sum of squares
        double l2Norm = Math.Sqrt(sumOfSquares);

        return l2Norm;
    }

### Temporal error estimation
To evaluate a temporal error, one reconstructs the state as a polynomial in time based on the endpoint states and velocities. With these 4 information, a cubic polynomial $\mathbf{U}(t)$ is uniquely defined and then projected to a lower-order polynomial in time $\mathbf{\tilde{U}}(t)$, here quadratic, with a $L_2$ projection over the time-step $t_n\rightarrow t_{n+1}$ with the constraint that the state at $t_n$ does not change. The temporal error is then the difference between the velocities, measured as fine-space residuals, at time node $t_{n+1}$ using the original state and the state from the lower-order temporal reconstruction
$$
\varepsilon_{time} = ||\mathbf{R}_{p+1}(\mathbf{I}_{p+1}^p \mathbf{U}_p^{n+1}) - \mathbf{R}_{p+1}(\mathbf{I}_{p+1}^p \mathbf{\tilde{U}}_p^{n+1})||.
$$
One finds for the lower-order temporal reconstructed state 
$$
\begin{split}
    \mathbf{\tilde{U}}_p^{n+1} = \mathbf{\tilde{U}}_p(\Delta t) &= \dfrac{17}{15}\mathbf{U}_p^{n+1}-\dfrac{2}{15}\mathbf{U}_p^n+\dfrac{\Delta t}{15}\left(\mathbf{M}_p^{-1}\mathbf{R}_p(\mathbf{U}_p^n)+\mathbf{M}^{-1}\mathbf{R}_p(\mathbf{U}_p^{n+1})\right).
\end{split}
$$

In [139]:
static double[] CalculateTemporalErrorElementWise(DGField[] Un,DGField[] Un1, IDifferentialOperator Op, double DeltaT)
{
    // First term
    var Un1_prol=ProlongateCoarseState(Un1);
    var Rp1Un1= EvalRes_toDGField(Un1_prol,Op);

    // Second term
    var UPtilde = EvaluateLowerOrderTemporalReconstruction(Un,Un1, DeltaT, Op);
    var UPtilde_prol= ProlongateCoarseState(UPtilde);
    var sec_term=EvalRes_toDGField(UPtilde_prol, Op);

    // Assemble the element-wise temporal error
    int nCells= Un[0].Basis.GridDat.iGeomCells.Count;
    var epsilonTime =new double[nCells];
    BitArray cells= new BitArray(Un[0].Basis.GridDat.iGeomCells.Count);

    for (int cell=0;cell<nCells; cell++){ //loop over all nCells
        cells[cell]=true;
        var cm= new CellMask(Un[0].Basis.GridDat,cells);
        for (int iField=0;iField< Un.Length;iField++){ //loop over Fields
            epsilonTime[cell]=Rp1Un1[iField].L2Error(sec_term[iField],cm);
        };
        cells[cell]=false;
    }
    return epsilonTime;
}

static DGField[] EvaluateLowerOrderTemporalReconstruction(DGField[] Un,DGField[] Un1, double DeltaT, IDifferentialOperator Op)
{
    var UPtilde = Un.CloneAs();
    for (int iField=0;iField< Un.Length;iField++){
        UPtilde[iField]=Un[iField].CloneAs();
        UPtilde[iField].Clear();
    }
    var U_tilde_vec= new CoordinateVector(UPtilde);
    double factor1 = 17.0 / 15.0;
    double factor2 = -2.0 / 15.0;
    U_tilde_vec.Acc(factor1,new CoordinateVector(Un1));
    U_tilde_vec.Acc(factor2,new CoordinateVector(Un));
    double factor3 = DeltaT / 15.0;
    var RpUn=EvalRes(Un,Op);
    var RpUn1=EvalRes(Un1,Op);
    
    //DG Case
    if(Un is SinglePhaseField[] uSF){ // no Mass Matrix mult needed
        U_tilde_vec.Acc(factor3,RpUn);
        U_tilde_vec.Acc(factor3,RpUn1);
    // XDG Case
    }else if(Un is XDGField[] Ux){
        throw new NotImplementedException("not implemented yet");
    }else {
        throw new NotImplementedException("field type not supported");
    }
    return UPtilde;
    //return Un1;
}
static double[] CalculateTemporalErrorFraction(double[] TempErr_elem,double[] SpatErr_elem ){
    int nCells=TempErr_elem.Length;
    double[] TEF=new double[nCells];
    for (int cell=0;cell<nCells; cell++){ 
        TEF[cell]=TempErr_elem[cell]/(TempErr_elem[cell] +SpatErr_elem[cell]);
    }
    return TEF;
}
static double CalculateTotalTemporalErrorFraction(DGField[] Un,DGField[] Un1, IDifferentialOperator Op, double DeltaT ){
    var TempErr_elem=CalculateTemporalErrorElementWise(Un,Un1,Op,DeltaT);
    var SpatErr_elem=CalculateSpatialErrorElementwise(Un1,Op);
    var TEF= CalculateTemporalErrorFraction(TempErr_elem,SpatErr_elem);
    return TEF.Average();
}


In [141]:
var nC=2; var t0=1.0;var t1=1.1;
var gridTest = Grid2D.Cartesian2DGrid(GenericBlas.Linspace(0.0,2.0,nC+1), GenericBlas.Linspace(0.0,2.0,nC+1));
var u0 = new SinglePhaseField(new Basis(gridTest, 2), "u");
var u1 = new SinglePhaseField(new Basis(gridTest, 2), "u");
u0.ProjectField((double[] X) => MyGlobals.uExact(X,t0));
u1.ProjectField((double[] X) => MyGlobals.uExact(X,t1));
var U0 =new SinglePhaseField[1]; U0[0]=u0;
var U1 =new SinglePhaseField[1]; U1[0]=u1;
//CalculateTemporalErrorElementWise(U0,U1,Op,t1-t0)
CalculateTotalTemporalErrorFraction(U0,U1,Op,t1-t0);
CalculateTemporalErrorElementWise(U0,U1,Op,t1-t0)


[ 140.64055997225682, 512.1659585344939, 1137.337330613406, 1338.4045311239097 ]

In [135]:
CalculateSpatialErrorElementwise(U1,Op)

[ 4.363964454917177, 72.71355959531232, 141.05102603459, 143.09180905876792 ]

In [136]:
CalculateTemporalErrorFraction(CalculateTemporalErrorElementWise(U0,U1,Op,t1-t0),CalculateSpatialErrorElementwise(U1,Op))

[ 0.9699046324784927, 0.8756777125179233, 0.8896649634666312, 0.9034139976065524 ]

In [62]:
    public (DGField[] Un1, List<double> TimeSteps)  AdaptiveTimeIntegrate(DifferentialOperator Op,DGField[] U0, double EndTime, double dtStart, TimeSteppingScheme tsScheme = TimeSteppingScheme.ESDIRK_64)
    {
        List<double> TimeSteps= new List<double>();
        double f_limit = 0.6;
        double r_max = 1.5;
        double r = 2.0;
        int tMax=1000;
        var Un1=CloneDGArray(U0);
        var IterationResiduals=CloneDGArray(U0);
        var Timestepper = new XdgTimestepping(Op, Un1, IterationResiduals, tsScheme);

        var currentTime=0.0;var currentTime_old=0.0;
        int NoOfTimeSteps = 0;
        double nextDt=dtStart;
        var Un1_vec=new CoordinateVector(Un1);
        while(currentTime+nextDt<EndTime && NoOfTimeSteps<tMax)
        {   
            //backup 
            var Un0 = CloneDGArray(Un1);

            //do timestep
            currentTime_old=currentTime;
            Timestepper.Solve(currentTime, nextDt);
            
            //calculate error estimator
            double f_time = CalculateTotalTemporalErrorFraction(Un0, Un1, Op, nextDt);
            double f = f_time / (1.0 - f_time);

            //decide what to do next
            if (f_time > f_limit)
            {
                // Redo the time step with DeltaT / 2
                Timestepper.LsTrk.PopStacks();
                var Un0_vec=new CoordinateVector(Un0);
                Un1_vec.Clear();
                Un1_vec.SetV(Un0_vec);
                nextDt=nextDt / 2.0;
                //Console.WriteLine($"f_time={f_time}, reducing ts to {nextDt}");
            }
            else
            {
                //add Ts
                currentTime=currentTime+nextDt;
                TimeSteps.Add(nextDt);
                // Set the next time step size
                nextDt = Math.Min(r_max,Math.Pow(f, -1.0 / r)) * nextDt;
                //Console.WriteLine($"f_time={f_time}, changing ts to {nextDt}");
                NoOfTimeSteps++;
            }
        }
        if( NoOfTimeSteps<tMax){
            Console.WriteLine("NoOfTimeSteps exceeded tMax");
        }
        if(currentTime<EndTime){
            Timestepper.Solve(currentTime, EndTime-currentTime);
        }
        return (Un1,TimeSteps);
    }
    DGField[] CloneDGArray(DGField[] u){
        var u_clone= u.CloneAs();
        for(int iField=0;iField<u.Length;iField++){
            u_clone[iField]=u[iField].CloneAs();
        }
        return u_clone;
    }

In [68]:
//var dtS = new double[] {1,0.5,0.25,0.125,0.0625,0.03175,0.015875,0.0079375};
bool debug=false;
var dtS = new double[] {1,0.5,0.25,0.125,0.0625};
var dtStartS = new double[] { 0.25 /*, 64, 128, 256, 512, 1028*/ };
var NCells = new double[] {4};
//var tsSchemes= new TimeSteppingScheme[] {TimeSteppingScheme.ESDIRK_64,TimeSteppingScheme.ESDIRK_32};
// var NCells = new double[] {4,8,16,32,64,128};
// var dtS = new double[] {1,0.5};
var tsSchemes= new TimeSteppingScheme[] {TimeSteppingScheme.ESDIRK_64};
var results = new List<Tuple<string,double[,]>>();
var solutions = new List<DGField>();
var TimestepS = new List<List<double>>();
//Grid and initial value
foreach(int nCells in NCells){
    Grid2D grid;
    if(!MyGlobals.isHeat){
        grid = Grid2D.Cartesian2DGrid(GenericBlas.Linspace(0.0,2.0,nCells+1), GenericBlas.Linspace(0.0,2.0,nCells+1));
    }else{
        grid = Grid2D.Cartesian2DGrid(GenericBlas.Linspace(-1.0,1.0,nCells+1), GenericBlas.Linspace(-1.0,1.0,nCells+1));
    }
    
    var u0 = new SinglePhaseField(new Basis(grid, 2), "u");
    u0.ProjectField((double[] X) => MyGlobals.uExact(X,0));
    //u0.ProjectField((double[] X) => Math.Cos(X[0]*Math.PI*0.5)*Math.Cos(X[1]*Math.PI*0.5));
    for(int ischeme=0;ischeme<tsSchemes.Length;ischeme++){
        var scheme=tsSchemes[ischeme];
        solutions = new List<DGField>();
        TimestepS = new List<List<double>>();
        Console.WriteLine("   Starting Compuation: " + scheme.ToString()+": nCells=" +grid.NumberOfCells);

        //### This loop uses a fixed timestep prescribed on the basis fo NoOfTimestepS
        for(int i = 0; i < dtStartS.Length; i++) {
            Console.WriteLine("   --------------  computing AdaptiveTimeStep using dtStart: " + dtStartS[i]);
            (var u1,List<double> TimeSteps)  = AdaptiveTimeIntegrate(Op,new SinglePhaseField[] {u0}, MyGlobals.endTime, dtStartS[i],scheme);
            //Tecplot("u1." + i, u1);
            solutions.Add(u1[0]);
            TimestepS.Add(TimeSteps);
        }

        if(debug){
            //######## some debugging ####################
            //compute res
            var Ev= Op.GetEvaluatorEx(new List<DGField>(){u1},null,new CoordinateMapping(u1));
            var residual=u1.CloneAs(); residual.Identification="residual";
            Ev.Evaluate(1.0,0.0,new CoordinateVector(residual));
            
            //diff to exact sol
            var diff=u1.CloneAs(); diff.Identification="diff_ex";
            diff.ProjectField((double[] X) => MyGlobals.uExact(X,MyGlobals.endTime));
            diff.Acc(-1.0,u1);

            //exact soll
            var uex=u1.CloneAs(); uex.Identification="u_ex";
            uex.ProjectField((double[] X) => MyGlobals.uExact(X,MyGlobals.endTime));

            //plot
            Tecplot("u1." + ((ischeme+1)*100+nCells), new DGField[] {u1,residual,diff,uex});
        }

        //Computing the error against the exact solution:
        double[,] ErrEx = new double[solutions.Count,2];
        for(int i = 0; i <solutions.Count; i++) {
            ErrEx[i,0] = solutions[i].L2Error(uExEq);
            ErrEx[i,1] = TimestepS[i].Average();
            //ErrEx[i]=residual.L2Norm();
        }
        results.Add(new Tuple<string,double[,]>(scheme.ToString()+ ": nCells=" +grid.NumberOfCells,ErrEx));
    }
}



In [69]:
TimestepS[0].Count

442

In [70]:
results[0]

Item1,ESDIRK_64: nCells=16
Item2,"[ 0.0329260254894481, 0.00452399465875922 ]"
